In [ ]:
# python packages
from sys import stdout
import pandas as pd # allows us to use dataframes
import numpy as np
import math
import matplotlib.pyplot as plt # allows us to make graphs
# below prevents warnings from outdated Pandas append command clogging output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib notebook

In [ ]:
# Cell below fixes scrolling issues for long outputs

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

In [ ]:
# initial conditions
RT = 8.314 * 298.15
Keq = 0.156

# adjustable parameter dc
dc = 1e-7

# initial concentrations and stoichiometric coefficient
concA = 0.15 ; nuA = -1
concB = 0.05 ; nuB = 1

# create a data frame for plotting
Data = {'Cycle': [0],
        '[A]': [concA],
        '[B]': [concB],
       }
df = pd.DataFrame(Data, columns=['Cycle','[A]','[B]'])

# print the data header
print("{:<30}{:<12}{:<12}{:<12}{:<12}".format("","[A]","[B]", "Q", "Keq"))

# reaction quotient ...
Q = concB / (concA*concA)
# print the first set of data
print("{:<30}{:<12}{:<12}{:<12}{:<12}".format("FIRST REACTION QUOTIENT","{:.5f}".format(concA),"{:.5f}".format(concB), "{:.5f}".format(Q), Keq))

# ... and driving force
force = RT * math.log(Keq / Q)

# infinitesimal change of the concentrations
# don't forget the stoichiometric coefficient
dn = force * dc
concA = concA + dn * nuA
concB = concB + dn * nuB

# NEW reaction quotient ...
Q = concB / (concA*concA)
# print the new set of data
print("{:<30}{:<12}{:<12}{:<12}{:<12}".format("NEW REACTION QUOTIENT","{:.5f}".format(concA),"{:.5f}".format(concB), "{:.5f}".format(Q), Keq))

# let's now make 1000 cycles
nCycles = 1000
df = pd.DataFrame(Data, columns=['Cycle','[A]','[B]'])
for icycle in range(0,nCycles):
    force = RT * math.log(Keq / Q)
    dn = force * dc
    concA = concA + dn * nuA
    concB = concB + dn * nuB
    Q = concB / (concA*concA)

    # write the concentrations on the screen
    if icycle%100 == 0:
        print("{:<30}{:<12}{:<12}{:<12}{:<12}".format("Cycle "+str(icycle),"{:.5f}".format(concA),"{:.5f}".format(concB), "{:.5f}".format(Q), Keq))
        
    # append data to dataframe for plotting
    if icycle%10 == 0:
        df = df.append({'Cycle' : icycle,
                        '[A]' : concA,
                        '[B]' : concB},
                        ignore_index = True)
        
print("","FULL DATAFRAME",df,sep="\n")

In [ ]:
# set the x variable as the "Cycle" column from the dataframe
x = df["Cycle"]
# set the y1 variable as the "[A]" column from the dataframe
y1 = df["[A]"]
# set the y2 variable as the "[B]" column from the dataframe
y2 = df["[B]"]

plt.xlabel("Number of Cycles")
plt.ylabel("Concentration")
plt.plot(x,y1,label="[A]")
plt.plot(x,y2,label="[B]")
plt.legend()

### Critical thinking questions
1. Verify that the final equilibrium concentrations are independent of the starting conditions, provided that $[A]+[B]$ is constant
2. What is the effect of changing the values of $dc$ or $nCycles$?
3. Is the value of RT important?